In [ ]:
import glob
import numpy as np
import os
import time
from collections import Counter
from multiprocessing import Pool, cpu_count

# Function to read sequences from files with each line as a single sequence
def read_sequences(filename):
    sequences = []
    with open(filename, 'r') as file:
        for line in file:
            sequence = line.strip()
            if sequence:
                sequences.append(sequence)
    return sequences

# Function to calculate dinucleotide perplexity and GC content in a single pass
def calculate_perplexity_and_gc(seq, window_size=10):
    seq_len = len(seq)
    if seq_len < window_size:
        return np.array([]), np.array([])

    num_windows = seq_len - window_size + 1
    perplexities = np.zeros(num_windows)
    gc_percentages = np.zeros(num_windows)

    dinucleotides = [seq[i:i + 2] for i in range(seq_len - 1)]

    for i in range(num_windows):
        window_dinucleotides = dinucleotides[i:i + window_size - 1]
        dinucleotide_counts = Counter(window_dinucleotides).values()
        probabilities = np.array(list(dinucleotide_counts)) / (window_size - 1)

        entropy = -np.sum(probabilities * np.log2(probabilities))
        perplexities[i] = 2 ** entropy

        window = seq[i:i + window_size]
        gc_count = window.count('G') + window.count('C')
        gc_percentages[i] = (gc_count / window_size) * 100

    return perplexities, gc_percentages

# Function to save results to a text file
def save_results_to_text(filename, x_values, avg_perplexity, avg_gc_content):
    output_filename = os.path.splitext(filename)[0] + "_results.txt"
    np.savetxt(
        output_filename,
        np.column_stack((x_values, avg_perplexity, avg_gc_content)),
        fmt="%d\t%.4f\t%.2f",
        header="Position\tPerplexity\tGC Content (%)",
        comments=""
    )
    print(f"Results saved to {output_filename}")

# Function to process each file and calculate averages
def process_file(file):
    sequences = read_sequences(file)
    if not sequences:
        print(f"No valid sequences found in {file}.")
        return

    window_size = 10
    num_positions = len(sequences[0]) - window_size + 1
    total_perplexities = np.zeros(num_positions)
    total_gc_content = np.zeros(num_positions)

    for sequence in sequences:
        perplexities, gc_content = calculate_perplexity_and_gc(sequence, window_size)
        total_perplexities += perplexities
        total_gc_content += gc_content

    sequence_count = len(sequences)
    avg_perplexity = total_perplexities / sequence_count
    avg_gc_content = total_gc_content / sequence_count
    x_values = np.arange(1, num_positions + 1)

    save_results_to_text(file, x_values, avg_perplexity, avg_gc_content)

# Function to process multiple files in parallel
def process_txt_files(pattern):
    files = glob.glob(pattern)
    if not files:
        print(f"No files found matching pattern: {pattern}")
        return

    start_time = time.time()

    # Use multiprocessing to process files simultaneously
    with Pool(cpu_count()) as pool:
        pool.map(process_file, files)

    elapsed_time = time.time() - start_time
    minutes, seconds = divmod(elapsed_time, 60)
    print(f"Time taken: {int(minutes)} minutes and {seconds:.2f} seconds")

# Process all files ending with _1000.txt in the current folder
process_txt_files("*_1000")

In [ ]:
import glob
import numpy as np
import os
import time
from collections import Counter
from multiprocessing import Pool

# Function to read sequences from files with each line as a single sequence
def read_sequences(filename):
    sequences = []
    with open(filename, 'r') as file:
        for line in file:
            sequence = line.strip()
            if sequence:
                sequences.append(sequence)
    return sequences

# Function to calculate dinucleotide perplexity and GC content in a single pass
def calculate_perplexity_and_gc(seq, window_size=10):
    seq_len = len(seq)
    if seq_len < window_size:
        return np.array([]), np.array([])

    num_windows = seq_len - window_size + 1
    perplexities = np.zeros(num_windows)
    gc_percentages = np.zeros(num_windows)

    dinucleotides = [seq[i:i + 2] for i in range(seq_len - 1)]

    for i in range(num_windows):
        window_dinucleotides = dinucleotides[i:i + window_size - 1]
        dinucleotide_counts = Counter(window_dinucleotides).values()
        probabilities = np.array(list(dinucleotide_counts)) / (window_size - 1)

        entropy = -np.sum(probabilities * np.log2(probabilities))
        perplexities[i] = 2 ** entropy

        window = seq[i:i + window_size]
        gc_count = window.count('G') + window.count('C')
        gc_percentages[i] = (gc_count / window_size) * 100

    return perplexities, gc_percentages

# Function to save results to a text file
def save_results_to_text(filename, x_values, avg_perplexity, avg_gc_content):
    output_filename = os.path.splitext(filename)[0] + "_results.txt"
    np.savetxt(
        output_filename,
        np.column_stack((x_values, avg_perplexity, avg_gc_content)),
        fmt="%d\t%.4f\t%.2f",
        header="Position\tPerplexity\tGC Content (%)",
        comments=""
    )
    print(f"Results saved to {output_filename}")

# Function to process each file and calculate averages
def process_file(file):
    print(f"Processing file: {file}")  # Print the file name being processed
    sequences = read_sequences(file)
    if not sequences:
        print(f"No valid sequences found in {file}.")
        return

    window_size = 10
    num_positions = len(sequences[0]) - window_size + 1
    total_perplexities = np.zeros(num_positions)
    total_gc_content = np.zeros(num_positions)

    for sequence in sequences:
        perplexities, gc_content = calculate_perplexity_and_gc(sequence, window_size)
        total_perplexities += perplexities
        total_gc_content += gc_content

    sequence_count = len(sequences)
    avg_perplexity = total_perplexities / sequence_count
    avg_gc_content = total_gc_content / sequence_count
    x_values = np.arange(1, num_positions + 1)

    save_results_to_text(file, x_values, avg_perplexity, avg_gc_content)

# Function to process multiple files in batches, limited to a maximum of 60 files at a time
def process_txt_files(pattern):
    files = glob.glob(pattern)
    if not files:
        print(f"No files found matching pattern: {pattern}")
        return

    start_time = time.time()
    # Process files in batches of up to 60 (to comply with Windows limitations)
    batch_size = 60
    for i in range(0, len(files), batch_size):
        batch_files = files[i:i + batch_size]
        with Pool(min(len(batch_files), 60)) as pool:
            pool.map(process_file, batch_files)

    elapsed_time = time.time() - start_time
    minutes, seconds = divmod(elapsed_time, 60)
    print(f"Time taken: {int(minutes)} minutes and {seconds:.2f} seconds")

# Process all files ending with _1000.txt in the current folder
process_txt_files("*_1000")


In [ ]:
import glob
import numpy as np
import os
import time
from collections import Counter

# Function to read sequences from files with each line as a single sequence
def read_sequences(filename):
    with open(filename, 'r') as file:
        return [line.strip() for line in file if line.strip()]

# Optimized function to calculate dinucleotide perplexity for a given sequence
def calculate_perplexity(seq):
    # Generate dinucleotides for the sequence
    dinucleotides = [seq[i:i + 2] for i in range(len(seq) - 1)]
    dinucleotide_counts = Counter(dinucleotides)
    
    # Get probabilities as a NumPy array for fast computation
    total_count = sum(dinucleotide_counts.values())
    probabilities = np.array(list(dinucleotide_counts.values())) / total_count

    # Calculate entropy and perplexity
    entropy = -np.sum(probabilities * np.log2(probabilities))
    perplexity = 2 ** entropy
    return perplexity

# Function to process a file and calculate perplexities for the specified regions
def process_file(file):
    print(f"Processing file: {file}")
    sequences = read_sequences(file)
    if not sequences:
        print(f"No valid sequences found in {file}.")
        return

    # Define regions of interest
    regions = [(100, 200), (400, 500), (700, 800)]
    region_perplexities = {region: [] for region in regions}

    for sequence in sequences:
        # Ensure the sequence is long enough for all regions
        if len(sequence) >= max(region[1] for region in regions):
            # Calculate mean perplexities for each region
            for region in regions:
                region_seq = sequence[region[0]:region[1]]
                region_perplexity = calculate_perplexity(region_seq)
                region_perplexities[region].append(region_perplexity)

    # Calculate and print percentages where each region has the lowest perplexity
    lower_count = {region: 0 for region in regions}
    total_sequences = len(sequences)

    for i in range(total_sequences):
        perplexities = [region_perplexities[region][i] for region in regions]
        min_perplexity = min(perplexities)
        for region in regions:
            if region_perplexities[region][i] == min_perplexity:
                lower_count[region] += 1

    # Calculate and display percentages
    for region in regions:
        percentage = (lower_count[region] / total_sequences) * 100
        print(f"Percentage of sequences where Region {region} is lowest: {percentage:.2f}%")

# Function to process multiple files one at a time
def process_txt_files(pattern):
    files = glob.glob(pattern)
    if not files:
        print(f"No files found matching pattern: {pattern}")
        return

    start_time = time.time()
    for file in files:
        process_file(file)

    elapsed_time = time.time() - start_time
    minutes, seconds = divmod(elapsed_time, 60)
    print(f"Time taken: {int(minutes)} minutes and {seconds:.2f} seconds")

# Process all files ending with _1000.txt in the current folder
process_txt_files("*_1000")


Processing file: Archaea_Euryarchaeota_Archaeoglobi_Archaeoglobus_fulgidus_NC_000917_1000
Percentage of sequences where Region (100, 200) is lowest: 26.84%
Percentage of sequences where Region (400, 500) is lowest: 48.51%
Percentage of sequences where Region (700, 800) is lowest: 24.65%
Processing file: Archaea_Euryarchaeota_Archaeoglobi_Archaeoglobus_profundus_NC_013741_1000
Percentage of sequences where Region (100, 200) is lowest: 30.43%
Percentage of sequences where Region (400, 500) is lowest: 42.49%
Percentage of sequences where Region (700, 800) is lowest: 27.08%
Processing file: Archaea_Euryarchaeota_Archaeoglobi_Archaeoglobus_sulfaticallidus_NC_021169_1000
Percentage of sequences where Region (100, 200) is lowest: 23.75%
Percentage of sequences where Region (400, 500) is lowest: 53.91%
Percentage of sequences where Region (700, 800) is lowest: 22.35%
Processing file: Archaea_Euryarchaeota_Archaeoglobi_Archaeoglobus_veneficus_NC_015320_1000
Percentage of sequences where Region 

In [2]:
import glob
import numpy as np
import pandas as pd
import time
from collections import Counter

# Function to read sequences from files with each line as a single sequence
def read_sequences(filename):
    with open(filename, 'r') as file:
        return [line.strip() for line in file if line.strip()]

# Optimized function to calculate dinucleotide perplexity for a given sequence
def calculate_perplexity(seq):
    # Generate dinucleotides for the sequence
    dinucleotides = [seq[i:i + 2] for i in range(len(seq) - 1)]
    dinucleotide_counts = Counter(dinucleotides)
    
    # Get probabilities as a NumPy array for fast computation
    total_count = sum(dinucleotide_counts.values())
    probabilities = np.array(list(dinucleotide_counts.values())) / total_count

    # Calculate entropy and perplexity
    entropy = -np.sum(probabilities * np.log2(probabilities))
    perplexity = 2 ** entropy
    return perplexity

# Function to process a file and calculate perplexities for the specified regions
def process_file(file):
    sequences = read_sequences(file)
    if not sequences:
        print(f"No valid sequences found in {file}.")
        return None

    # Define 80-mer regions of interest
    regions = [(1, 81), (400, 480), (900, 980)]  # Including promoter and other regions
    region_perplexities = {region: [] for region in regions}

    for sequence in sequences:
        # Ensure the sequence is long enough for all regions
        if len(sequence) >= max(region[1] for region in regions):
            # Calculate mean perplexities for each region
            for region in regions:
                region_seq = sequence[region[0]:region[1]]
                region_perplexity = calculate_perplexity(region_seq)
                region_perplexities[region].append(region_perplexity)

    # Calculate percentages where each region has the lowest perplexity
    lower_count = {region: 0 for region in regions}
    total_sequences = len(sequences)

    for i in range(total_sequences):
        perplexities = [region_perplexities[region][i] for region in regions]
        min_perplexity = min(perplexities)
        for region in regions:
            if region_perplexities[region][i] == min_perplexity:
                lower_count[region] += 1

    # Calculate percentages
    percentages = [(lower_count[region] / total_sequences) * 100 for region in regions]
    return [file] + percentages

# Function to process multiple files one at a time and save results to Excel
def process_txt_files(pattern):
    files = glob.glob(pattern)
    if not files:
        print(f"No files found matching pattern: {pattern}")
        return

    start_time = time.time()
    results = []

    for file in files:
        print(f"Processing file: {file}")
        result = process_file(file)
        if result:
            results.append(result)

    # Save results to a DataFrame and export to Excel
    df = pd.DataFrame(results, columns=['File', 'Region1%', 'Region2%', 'Region3%'])
    output_file = "perplexity_results.xlsx"
    df.to_excel(output_file, index=False)
    print(f"Results saved to {output_file}")

    elapsed_time = time.time() - start_time
    minutes, seconds = divmod(elapsed_time, 60)
    print(f"Time taken: {int(minutes)} minutes and {seconds:.2f} seconds")

# Process all files ending with _1000.txt in the current folder
process_txt_files("*_1000")

Processing file: Archaea_Euryarchaeota_Archaeoglobi_Archaeoglobus_fulgidus_NC_000917_1000
Processing file: Archaea_Euryarchaeota_Archaeoglobi_Archaeoglobus_profundus_NC_013741_1000
Processing file: Archaea_Euryarchaeota_Archaeoglobi_Archaeoglobus_sulfaticallidus_NC_021169_1000
Processing file: Archaea_Euryarchaeota_Archaeoglobi_Archaeoglobus_veneficus_NC_015320_1000
Processing file: Archaea_Euryarchaeota_Archaeoglobi_Ferroglobus_placidus_NC_013849_1000
Processing file: Archaea_Euryarchaeota_Diaforarchaea_group_Aciduliprofundum_boonei_NC_013926_1000
Processing file: Archaea_Euryarchaeota_Diaforarchaea_group_Candidatus_Methanomassiliicoccus_intestinalis_NC_021353_1000
Processing file: Archaea_Euryarchaeota_Diaforarchaea_group_Candidatus_Methanomethylophilus_NC_020913_1000
Processing file: Archaea_Euryarchaeota_Diaforarchaea_group_Ferroplasma_acidarmanus_NC_021592_1000
Processing file: Archaea_Euryarchaeota_Diaforarchaea_group_Picrophilus_torridus_NC_005877_1000
Processing file: Archaea_E

PermissionError: [Errno 13] Permission denied: 'perplexity_results.xlsx'

In [1]:
import glob
import numpy as np
import os
import time
from collections import Counter
import pandas as pd

# Function to read sequences from files with each line as a single sequence
def read_sequences(filename):
    with open(filename, 'r') as file:
        return [line.strip() for line in file if line.strip()]

# Optimized function to calculate dinucleotide perplexity for a given sequence
def calculate_perplexity(seq):
    dinucleotides = [seq[i:i + 2] for i in range(len(seq) - 1)]
    dinucleotide_counts = Counter(dinucleotides)
    total_count = sum(dinucleotide_counts.values())
    probabilities = np.array(list(dinucleotide_counts.values())) / total_count
    entropy = -np.sum(probabilities * np.log2(probabilities))
    perplexity = 2 ** entropy
    return perplexity

# Function to process a file and calculate perplexities for the specified regions
def process_file(file):
    print(f"Processing file: {file}")
    sequences = read_sequences(file)
    if not sequences:
        print(f"No valid sequences found in {file}.")
        return None

    # Define regions of interest
    regions = [(200, 280), (400, 480), (700, 780)]
    region_perplexities = {region: [] for region in regions}

    for sequence in sequences:
        # Ensure the sequence is long enough for all regions
        if len(sequence) >= max(region[1] for region in regions):
            # Calculate perplexities for each region
            for region in regions:
                region_seq = sequence[region[0]:region[1]]
                region_perplexity = calculate_perplexity(region_seq)
                region_perplexities[region].append(region_perplexity)

    # Calculate percentages where each region has the lowest median perplexity
    lower_count = {region: 0 for region in regions}
    total_sequences = len(sequences)

    for i in range(total_sequences):
        medians = {region: np.median(region_perplexities[region][i:i+1]) for region in regions}
        min_median_region = min(medians, key=medians.get)
        lower_count[min_median_region] += 1

    # Calculate and display percentages
    percentages = {region: (lower_count[region] / total_sequences) * 100 for region in regions}
    for region, percentage in percentages.items():
        print(f"Percentage of sequences where Region {region} has the lowest median perplexity: {percentage:.2f}%")

    return [file, percentages[regions[0]], percentages[regions[1]], percentages[regions[2]]]

# Function to process multiple files one at a time and save results to Excel
def process_txt_files(pattern):
    files = glob.glob(pattern)
    if not files:
        print(f"No files found matching pattern: {pattern}")
        return

    start_time = time.time()
    results = []

    for file in files:
        result = process_file(file)
        if result:
            results.append(result)

    # Convert results to a DataFrame and save to Excel
    df = pd.DataFrame(results, columns=['File', 'Region 200-280 (%)', 'Region 400-480 (%)', 'Region 700-780 (%)'])
    output_file = "perplexity_percentage_results.xlsx"
    df.to_excel(output_file, index=False)
    print(f"Results saved to {output_file}")

    elapsed_time = time.time() - start_time
    minutes, seconds = divmod(elapsed_time, 60)
    print(f"Time taken: {int(minutes)} minutes and {seconds:.2f} seconds")

# Process all files ending with _1000.txt in the current folder
process_txt_files("*_1000")


Processing file: Archaea_Euryarchaeota_Archaeoglobi_Archaeoglobus_fulgidus_NC_000917_1000
Percentage of sequences where Region (200, 280) has the lowest median perplexity: 27.01%
Percentage of sequences where Region (400, 480) has the lowest median perplexity: 47.35%
Percentage of sequences where Region (700, 780) has the lowest median perplexity: 25.64%
Processing file: Archaea_Euryarchaeota_Archaeoglobi_Archaeoglobus_profundus_NC_013741_1000
Percentage of sequences where Region (200, 280) has the lowest median perplexity: 28.95%
Percentage of sequences where Region (400, 480) has the lowest median perplexity: 42.65%
Percentage of sequences where Region (700, 780) has the lowest median perplexity: 28.40%
Processing file: Archaea_Euryarchaeota_Archaeoglobi_Archaeoglobus_sulfaticallidus_NC_021169_1000
Percentage of sequences where Region (200, 280) has the lowest median perplexity: 25.37%
Percentage of sequences where Region (400, 480) has the lowest median perplexity: 51.06%
Percentage

In [1]:
import glob
import numpy as np
import os
import time
from collections import Counter
import pandas as pd

# Function to read sequences from files with each line as a single sequence
def read_sequences(filename):
    with open(filename, 'r') as file:
        return [line.strip() for line in file if line.strip()]

# Optimized function to calculate dinucleotide perplexity for a given sequence
def calculate_perplexity(seq):
    dinucleotides = [seq[i:i + 2] for i in range(len(seq) - 1)]
    dinucleotide_counts = Counter(dinucleotides)
    total_count = sum(dinucleotide_counts.values())
    probabilities = np.array(list(dinucleotide_counts.values())) / total_count
    entropy = -np.sum(probabilities * np.log2(probabilities))
    perplexity = 2 ** entropy
    return perplexity

# Function to calculate the top 10 most frequent 10-mers in a specific region
def top_10_mers(seq, mer_length=10):
    mers = [seq[i:i + mer_length] for i in range(len(seq) - mer_length + 1)]
    mer_counts = Counter(mers)
    top_mers = mer_counts.most_common(10)
    return ', '.join([mer[0] for mer in top_mers])

# Function to process a file and calculate perplexities for the specified regions
def process_file(file, file_index):
    print(f"Processing file {file_index}: {file}")
    sequences = read_sequences(file)
    if not sequences:
        print(f"No valid sequences found in {file}.")
        return None

    # Define regions of interest
    regions = [(100, 180), (400, 480), (800, 880)]
    region_perplexities = {region: [] for region in regions}
    top_10_mers_list = []

    for sequence in sequences:
        # Ensure the sequence is long enough for all regions
        if len(sequence) >= max(region[1] for region in regions):
            # Calculate perplexities for each region
            for region in regions:
                region_seq = sequence[region[0]:region[1]]
                region_perplexity = calculate_perplexity(region_seq)
                region_perplexities[region].append(region_perplexity)
            
            # Calculate top 10 most frequent 10-mers for the 400-480 region only
            top_10_mers_list.append(top_10_mers(sequence[400:480]))

    # Calculate percentages where each region has the lowest mean perplexity
    lower_count = {region: 0 for region in regions}
    total_sequences = len(sequences)

    for i in range(total_sequences):
        means = {region: np.mean(region_perplexities[region][i:i+1]) for region in regions}
        min_mean_region = min(means, key=means.get)
        lower_count[min_mean_region] += 1

    # Calculate percentages for each region
    percentages = {region: (lower_count[region] / total_sequences) * 100 for region in regions}
    for region, percentage in percentages.items():
        print(f"Percentage of sequences where Region {region} has the lowest mean perplexity: {percentage:.2f}%")

    # Extract top 10 motifs for output
    top_10_mers_combined = top_10_mers_list[0] if top_10_mers_list else ''

    # Add file index and top motifs to result
    return [file_index, file, percentages[regions[0]], percentages[regions[1]], percentages[regions[2]], top_10_mers_combined]

# Function to process multiple files one at a time and save results to Excel
def process_txt_files(pattern):
    files = glob.glob(pattern)
    if not files:
        print(f"No files found matching pattern: {pattern}")
        return

    start_time = time.time()
    results = []

    for file_index, file in enumerate(files, start=1):
        result = process_file(file, file_index)
        if result:
            results.append(result)

    # Convert results to a DataFrame and save to Excel
    df = pd.DataFrame(results, columns=['File Index', 'File', 'Region 100-180 (%)', 'Region 400-480 (%)', 'Region 800-880 (%)', 'Top 10-mers (400-480)'])
    output_file = "perplexity_percentage_results.xlsx"
    try:
        df.to_excel(output_file, index=False)
        print(f"Results saved to {output_file}")
    except PermissionError:
        print(f"PermissionError: Please close the file '{output_file}' if it is open and try again.")

    elapsed_time = time.time() - start_time
    minutes, seconds = divmod(elapsed_time, 60)
    print(f"Time taken: {int(minutes)} minutes and {seconds:.2f} seconds")

# Process all files ending with _1000.txt in the current folder
process_txt_files("*_1000")


Processing file 1: Archaea_Euryarchaeota_Archaeoglobi_Archaeoglobus_fulgidus_NC_000917_1000
Percentage of sequences where Region (100, 180) has the lowest mean perplexity: 28.29%
Percentage of sequences where Region (400, 480) has the lowest mean perplexity: 47.06%
Percentage of sequences where Region (800, 880) has the lowest mean perplexity: 24.65%
Processing file 2: Archaea_Euryarchaeota_Archaeoglobi_Archaeoglobus_profundus_NC_013741_1000
Percentage of sequences where Region (100, 180) has the lowest mean perplexity: 30.27%
Percentage of sequences where Region (400, 480) has the lowest mean perplexity: 42.76%
Percentage of sequences where Region (800, 880) has the lowest mean perplexity: 26.97%
Processing file 3: Archaea_Euryarchaeota_Archaeoglobi_Archaeoglobus_sulfaticallidus_NC_021169_1000
Percentage of sequences where Region (100, 180) has the lowest mean perplexity: 24.33%
Percentage of sequences where Region (400, 480) has the lowest mean perplexity: 52.19%
Percentage of sequen

In [ ]:
import glob
import numpy as np
import pandas as pd
from collections import Counter
import time

# Function to read sequences from files with each line as a single sequence
def read_sequences(filename):
    with open(filename, 'r') as file:
        return [line.strip() for line in file if line.strip()]

# Optimized function to calculate dinucleotide perplexity for a given sequence
def calculate_perplexity(seq):
    dinucleotides = [seq[i:i + 2] for i in range(len(seq) - 1)]
    dinucleotide_counts = Counter(dinucleotides)
    total_count = sum(dinucleotide_counts.values())
    probabilities = np.array(list(dinucleotide_counts.values())) / total_count
    entropy = -np.sum(probabilities * np.log2(probabilities))
    perplexity = 2 ** entropy
    return perplexity

# Function to calculate sliding-window perplexity or GC for each position
def sliding_window_analysis(seq, window_size=10, step=5, calc_gc=False):
    results = []
    for i in range(0, len(seq) - window_size + 1, step):
        window_seq = seq[i:i + window_size]
        if calc_gc:
            gc_count = window_seq.count('G') + window_seq.count('C')
            gc_percentage = (gc_count / window_size) * 100
            results.append(gc_percentage)
        else:
            results.append(calculate_perplexity(window_seq))
    return results

# Function to calculate positional averages across sequences
def calculate_positional_averages(sequences, calc_gc=False):
    max_len = max(len(seq) for seq in sequences)
    pos_sums = np.zeros((max_len,))
    counts = np.zeros((max_len,))

    for sequence in sequences:
        results = sliding_window_analysis(sequence, calc_gc=calc_gc)
        for pos, value in enumerate(results):
            pos_sums[pos] += value
            counts[pos] += 1

    avg_results = pos_sums / np.maximum(counts, 1)
    return avg_results[:len(results)]

# Function to analyze specific regions and save results
def analyze_regions(sequences, regions=[(100, 180), (400, 480), (800, 880)], threshold=0.3):
    region_stats = []
    criterion_counts = {region: 0 for region in regions}

    for seq in sequences:
        region_means = {}
        for region in regions:
            region_seq = seq[region[0]:region[1]]
            region_means[region] = np.mean(sliding_window_analysis(region_seq))

        for region in regions:
            if all(region_means[region] >= region_means[other] + threshold for other in regions if other != region):
                criterion_counts[region] += 1

        region_stats.append(region_means)

    # Calculate mean perplexity and percentage
    mean_stats = {region: np.mean([r[region] for r in region_stats]) for region in regions}
    percentages = {region: (criterion_counts[region] / len(sequences)) * 100 for region in regions}

    return mean_stats, percentages

# Function to process files and save results to Excel
def process_files(pattern):
    files = glob.glob(pattern)
    perplexity_results, gc_results = [], []
    region_results = []

    start_time = time.time()

    for file_index, file in enumerate(files, start=1):
        sequences = read_sequences(file)
        if not sequences:
            print(f"No valid sequences found in {file}.")
            continue

        # Positional averages
        avg_perplexity = calculate_positional_averages(sequences)
        avg_gc = calculate_positional_averages(sequences, calc_gc=True)

        # Save positional average results
        perplexity_results.append(pd.DataFrame({'Position': range(len(avg_perplexity)), 'Avg Perplexity': avg_perplexity}))
        gc_results.append(pd.DataFrame({'Position': range(len(avg_gc)), 'Avg GC%': avg_gc}))

        # Analyze regions for each file
        mean_stats, percentages = analyze_regions(sequences)
        region_results.append([file, *[mean_stats[region] for region in mean_stats], *[percentages[region] for region in percentages]])

    # Save results for each file in separate Excel files
    for i, (file, df_perplexity, df_gc) in enumerate(zip(files, perplexity_results, gc_results)):
        perplexity_file = f"{os.path.splitext(file)[0]}_perplexity.xlsx"
        gc_file = f"{os.path.splitext(file)[0]}_gc.xlsx"
        df_perplexity.to_excel(perplexity_file, index=False)
        df_gc.to_excel(gc_file, index=False)
        print(f"Saved: {perplexity_file} and {gc_file}")

    # Save region analysis to single Excel file
    df_regions = pd.DataFrame(region_results, columns=[
        'File', 'Mean Perplexity 100-180', 'Mean Perplexity 400-480', 'Mean Perplexity 800-880',
        '100-180 % Below Threshold', '400-480 % Below Threshold', '800-880 % Below Threshold'
    ])
    output_file = "region_analysis_results.xlsx"
    df_regions.to_excel(output_file, index=False)
    print(f"Region analysis results saved to {output_file}")

    elapsed_time = time.time() - start_time
    minutes, seconds = divmod(elapsed_time, 60)
    print(f"Total Time taken: {int(minutes)} minutes and {seconds:.2f} seconds")

# Process all files ending with _1000.txt in the current folder
process_files("*_1000")


In [2]:
import glob
import numpy as np
import pandas as pd
from collections import Counter
import os
import time

# Function to read sequences from files with each line as a single sequence
def read_sequences(filename):
    with open(filename, 'r') as file:
        return [line.strip() for line in file if line.strip()]

# Function to calculate dinucleotide perplexity for a given sequence
def calculate_perplexity(seq):
    dinucleotides = [seq[i:i + 2] for i in range(len(seq) - 1)]
    dinucleotide_counts = Counter(dinucleotides)
    total_count = sum(dinucleotide_counts.values())
    probabilities = np.array(list(dinucleotide_counts.values())) / total_count
    entropy = -np.sum(probabilities * np.log2(probabilities))
    perplexity = 2 ** entropy
    return perplexity

# Function to calculate sliding-window perplexity or GC for each position in a sequence
def sliding_window_analysis(seq, window_size=10, step=5, calc_gc=False):
    results = []
    for i in range(0, len(seq) - window_size + 1, step):
        window_seq = seq[i:i + window_size]
        if calc_gc:
            # Calculate GC content for each window
            gc_count = window_seq.count('G') + window_seq.count('C')
            gc_percentage = (gc_count / window_size) * 100
            results.append(gc_percentage)
        else:
            # Calculate perplexity for each window
            results.append(calculate_perplexity(window_seq))
    return results

# Function to calculate positional averages for perplexity or GC content across sequences
def calculate_positional_averages(sequences, window_size=10, step=5, calc_gc=False):
    max_len = max(len(seq) for seq in sequences)
    num_positions = (max_len - window_size) // step + 1
    pos_sums = np.zeros(num_positions)
    counts = np.zeros(num_positions)

    for sequence in sequences:
        results = sliding_window_analysis(sequence, window_size, step, calc_gc)
        for pos, value in enumerate(results):
            pos_sums[pos] += value
            counts[pos] += 1

    avg_results = pos_sums / np.maximum(counts, 1)
    return avg_results

# Function to analyze specified regions for perplexity and GC content
def analyze_regions(sequences, regions=[(100, 180), (400, 480), (800, 880)], threshold=0.3):
    region_means = []
    criterion_counts = {region: 0 for region in regions}

    for seq in sequences:
        region_stats = {}
        for region in regions:
            region_seq = seq[region[0]:region[1]]
            mean_perplexity = np.mean(sliding_window_analysis(region_seq))
            region_stats[region] = mean_perplexity

        for region in regions:
            if all(region_stats[region] <= region_stats[other] - threshold for other in regions if other != region):
                criterion_counts[region] += 1
        region_means.append(region_stats)

    # Calculate mean perplexity and percentage of sequences meeting criterion
    mean_stats = {region: np.mean([r[region] for r in region_means]) for region in regions}
    percentages = {region: (criterion_counts[region] / len(sequences)) * 100 for region in regions}

    return mean_stats, percentages

# Function to process files and save results
def process_files(pattern):
    files = glob.glob(pattern)
    perplexity_data, gc_data, region_results = [], [], []
    start_time = time.time()

    for file_index, file in enumerate(files, start=1):
        print(f"Processing file {file_index}: {file}")
        sequences = read_sequences(file)
        if not sequences:
            print(f"No valid sequences found in {file}.")
            continue

        # Calculate positional average perplexity and GC content
        avg_perplexity = calculate_positional_averages(sequences)
        avg_gc = calculate_positional_averages(sequences, calc_gc=True)

        # Append to positional averages lists with file name
        perplexity_data.append([os.path.basename(file)] + list(avg_perplexity))
        gc_data.append([os.path.basename(file)] + list(avg_gc))

        # Analyze regions for each file
        mean_stats, percentages = analyze_regions(sequences)
        region_results.append([os.path.basename(file), *[mean_stats[region] for region in mean_stats], *[percentages[region] for region in percentages]])

        # Save intermediate outputs to check progress
        if file_index % 5 == 0:
            pd.DataFrame(perplexity_data).to_csv("intermediate_perplexity_output.csv", index=False, header=False)
            pd.DataFrame(gc_data).to_csv("intermediate_gc_output.csv", index=False, header=False)

    # Save final positional average results
    perplexity_df = pd.DataFrame(perplexity_data)
    gc_df = pd.DataFrame(gc_data)
    perplexity_df.to_csv("perplexity_positional_output.csv", index=False, header=False)
    gc_df.to_csv("gc_positional_output.csv", index=False, header=False)
    print("Positional results saved to 'perplexity_positional_output.csv' and 'gc_positional_output.csv'")

    # Save region analysis results
    region_df = pd.DataFrame(region_results, columns=[
        'File', 'Mean Perplexity 100-180', 'Mean Perplexity 400-480', 'Mean Perplexity 800-880',
        '100-180 % Below Threshold', '400-480 % Below Threshold', '800-880 % Below Threshold'
    ])
    region_df.to_excel("region_analysis_results.xlsx", index=False)
    print("Region analysis results saved to 'region_analysis_results.xlsx'")

    elapsed_time = time.time() - start_time
    minutes, seconds = divmod(elapsed_time, 60)
    print(f"Total Time taken: {int(minutes)} minutes and {seconds:.2f} seconds")

# Process all files ending with _1000.txt in the current folder
process_files("*_1000")


Processing file 1: Archaea_Euryarchaeota_Archaeoglobi_Archaeoglobus_fulgidus_NC_000917_1000
Processing file 2: Archaea_Euryarchaeota_Archaeoglobi_Archaeoglobus_profundus_NC_013741_1000
Processing file 3: Archaea_Euryarchaeota_Archaeoglobi_Archaeoglobus_sulfaticallidus_NC_021169_1000
Processing file 4: Archaea_Euryarchaeota_Archaeoglobi_Archaeoglobus_veneficus_NC_015320_1000
Processing file 5: Archaea_Euryarchaeota_Archaeoglobi_Ferroglobus_placidus_NC_013849_1000
Processing file 6: Archaea_Euryarchaeota_Diaforarchaea_group_Aciduliprofundum_boonei_NC_013926_1000
Processing file 7: Archaea_Euryarchaeota_Diaforarchaea_group_Candidatus_Methanomassiliicoccus_intestinalis_NC_021353_1000
Processing file 8: Archaea_Euryarchaeota_Diaforarchaea_group_Candidatus_Methanomethylophilus_NC_020913_1000
Processing file 9: Archaea_Euryarchaeota_Diaforarchaea_group_Ferroplasma_acidarmanus_NC_021592_1000
Processing file 10: Archaea_Euryarchaeota_Diaforarchaea_group_Picrophilus_torridus_NC_005877_1000
Proce

KeyboardInterrupt: 

In [1]:
import glob
import numpy as np
import pandas as pd
from collections import Counter
import os
import time

# Function to read sequences from files with each line as a single sequence
def read_sequences(filename):
    with open(filename, 'r') as file:
        return [line.strip() for line in file if line.strip()]

# Function to calculate dinucleotide perplexity for a given sequence
def calculate_perplexity(seq):
    dinucleotides = [seq[i:i + 2] for i in range(len(seq) - 1)]
    counts = Counter(dinucleotides).values()
    probabilities = np.array(list(counts)) / sum(counts)
    entropy = -np.sum(probabilities * np.log2(probabilities))
    return 2 ** entropy

# Function to calculate sliding-window perplexity or GC for each position in a sequence
def sliding_window_analysis(seq, window_size=10, step=10, calc_gc=False):
    results = []
    for i in range(0, len(seq) - window_size + 1, step):
        window_seq = seq[i:i + window_size]
        if calc_gc:
            gc_percentage = (window_seq.count('G') + window_seq.count('C')) / window_size * 100
            results.append(gc_percentage)
        else:
            results.append(calculate_perplexity(window_seq))
    return results

# Function to calculate positional averages for perplexity or GC content across sequences
def calculate_positional_averages(sequences, window_size=10, step=10, calc_gc=False):
    max_len = max(len(seq) for seq in sequences)
    num_positions = (max_len - window_size) // step + 1
    pos_sums = np.zeros(num_positions)
    counts = np.zeros(num_positions)

    for sequence in sequences:
        results = sliding_window_analysis(sequence, window_size, step, calc_gc)
        for pos, value in enumerate(results):
            pos_sums[pos] += value
            counts[pos] += 1

    return pos_sums / np.maximum(counts, 1)  # Avoid division by zero

# Function to analyze specified regions for perplexity and GC content
def analyze_regions(sequences, regions=[(100, 180), (400, 480), (800, 880)], threshold=0.3):
    region_means = []
    criterion_counts = {region: 0 for region in regions}

    for seq in sequences:
        region_stats = {region: np.mean(sliding_window_analysis(seq[region[0]:region[1]], step=10)) for region in regions}
        region_means.append(region_stats)

        for region in regions:
            if all(region_stats[region] <= region_stats[other] - threshold for other in regions if other != region):
                criterion_counts[region] += 1

    mean_stats = {region: np.mean([r[region] for r in region_means]) for region in regions}
    percentages = {region: (criterion_counts[region] / len(sequences)) * 100 for region in regions}

    return mean_stats, percentages

# Function to process files and save results
def process_files(pattern):
    files = glob.glob(pattern)
    perplexity_data, gc_data, region_results = [], [], []
    start_time = time.time()

    for file_index, file in enumerate(files, start=1):
        print(f"Processing file {file_index}: {file}")
        sequences = read_sequences(file)
        if not sequences:
            print(f"No valid sequences found in {file}.")
            continue

        avg_perplexity = calculate_positional_averages(sequences)
        avg_gc = calculate_positional_averages(sequences, calc_gc=True)

        # Append to positional averages lists with file name
        perplexity_data.append([os.path.basename(file)] + list(avg_perplexity))
        gc_data.append([os.path.basename(file)] + list(avg_gc))

        # Analyze regions for each file
        mean_stats, percentages = analyze_regions(sequences)
        region_results.append([os.path.basename(file), *[mean_stats[region] for region in mean_stats], *[percentages[region] for region in percentages]])

    # Save final positional average results
    pd.DataFrame(perplexity_data).to_csv("perplexity_positional_output.csv", index=False, header=False)
    pd.DataFrame(gc_data).to_csv("gc_positional_output.csv", index=False, header=False)
    print("Positional results saved to 'perplexity_positional_output.csv' and 'gc_positional_output.csv'")

    # Save region analysis results
    region_df = pd.DataFrame(region_results, columns=[
        'File', 'Mean Perplexity 100-180', 'Mean Perplexity 400-480', 'Mean Perplexity 800-880',
        '100-180 % Below Threshold', '400-480 % Below Threshold', '800-880 % Below Threshold'
    ])
    region_df.to_excel("region_analysis_results.xlsx", index=False)
    print("Region analysis results saved to 'region_analysis_results.xlsx'")

    elapsed_time = time.time() - start_time
    minutes, seconds = divmod(elapsed_time, 60)
    print(f"Total Time taken: {int(minutes)} minutes and {seconds:.2f} seconds")

# Execute the function
process_files("*_1000")


Processing file 1: Archaea_Euryarchaeota_Archaeoglobi_Archaeoglobus_fulgidus_NC_000917_1000
Processing file 2: Archaea_Euryarchaeota_Archaeoglobi_Archaeoglobus_profundus_NC_013741_1000
Processing file 3: Archaea_Euryarchaeota_Archaeoglobi_Archaeoglobus_sulfaticallidus_NC_021169_1000
Processing file 4: Archaea_Euryarchaeota_Archaeoglobi_Archaeoglobus_veneficus_NC_015320_1000
Processing file 5: Archaea_Euryarchaeota_Archaeoglobi_Ferroglobus_placidus_NC_013849_1000
Processing file 6: Archaea_Euryarchaeota_Diaforarchaea_group_Aciduliprofundum_boonei_NC_013926_1000
Processing file 7: Archaea_Euryarchaeota_Diaforarchaea_group_Candidatus_Methanomassiliicoccus_intestinalis_NC_021353_1000
Processing file 8: Archaea_Euryarchaeota_Diaforarchaea_group_Candidatus_Methanomethylophilus_NC_020913_1000
Processing file 9: Archaea_Euryarchaeota_Diaforarchaea_group_Ferroplasma_acidarmanus_NC_021592_1000
Processing file 10: Archaea_Euryarchaeota_Diaforarchaea_group_Picrophilus_torridus_NC_005877_1000
Proce